# General fine-tuning of text generation models through instructions

В предыдущих семинарах мы в основном фокусировались на какой-то одной задаче - исправление опечаток, выделение сущностей, определение тональности, машинный перевод и т.п. Мы двигались от самых простых моделей к предобученным трансформерам, но подход в целом не менялся - мы брали модель и тренировали её решать нужную задачу, а когда нужно было решить новую задачу мы откладывали старую модель и тренировали новую. 

Один из последних быстро развивающихся трендов в NLP - решать множество задач 1 общей моделью. Если задуматься, то все к этому шло:

1) Self-supervised предобучение научились более менее стабильно масштабировать на огромный текстовые корпуса, что дало нам модели, в которых уже заложено очень широкое понимание языка
2) Fine-tuning предобученных моделей под конкретную задачу требует небольшое количество примеров (сотни или даже десятки
3) При правильных промтах предобученный модели могли решать даже задачи, которые они никогда не видели (zero-shot и in context learning)
4) Все возможные NLP задачи стали сводится к генерации текста (классификация - генерация одного токена, исправление опечаток - генерация исправленной последовательности, выделение сущности - генерация именованых сущностей нужного типа, даже генерация кода теперь решается просто генерацией)


Поэтому было вопросом времени, когда кто-то попробует обучить модель решать сразу все нужные задачи и у них получится.

Текущее топовое решение - дообучить модель на датесете разнообразных инструкций, которые соответствуют нужным задачам, а затем дотренировать новую модель с помощью Reinforcement Learning from Human Feedback (RLHF). Такой подход первым успешно применил OpenAI и результат можно наблюдать в ChatGPT (спойлер: результат очень хороший). Но OpenAI не опубликовал в открытом доступе ни модели, ни код ни какие-то технические описания их подхода. Поэтому сейчас болшАя часть исследовательского сообщества в NLP занимается тем, что пытается воспроизвести chatgpt по общем описаниям, которые раскрыл OpenAI. И очень многое уже получилось воспроизвести и буквально с каждым днем такие модели становятся меньше/дешевле и доступнее.

В этом семинаре мы попробуем дообучить модель на датасете инструкций. Но прежде чем переходить к этому, давайте посмотрим на две статьи (и модели), которые есть в открытом доступе и которые сильно повлияли на движение в сторону общих моделей.

In [17]:
# %pip install pandas transformers tokenizers datasets xformers

## T5

![](https://1.bp.blogspot.com/-o4oiOExxq1s/Xk26XPC3haI/AAAAAAAAFU8/NBlvOWB84L0PTYy9TzZBaLf6fwPGJTR0QCLcBGAsYHQ/s640/image3.gif)

Первая статья Т5 (Text-To-Text Transfer Transformer, https://arxiv.org/abs/1910.10683, Google Research, конец 2019 года) 
Это очень большая статья, в которой подробно исследовалась унификация различных NLP задач в задачу генерации. А также они попробовали много различных подходов к предобучению (в то время выходило очень много статей, которые как-то меняли self-supervised задачу и они попробовали много разных комбинаций, чтобы получить хорошую модель). В результате у них получилось несколько вариантов модели Т5 и всех их они выложили в открытый доступ. 
Еще в статье они пробовали тюнить модель под разные задачи, но по большей части все еще по отдельности. Если вы прочитаете статью или хотя бы описание fine-tuning экспериментов, то увидите, что на тот момент парадигма (1 модель - 1 задача) еще не изменилась. В своих экспериментах они пробовали тренироваться сразу под несколько задач, но у них было не достаточно много разнообразных задач и в итоге общая модель работала хуже на отдельных задачах, чем специфичные модели.

Но в открытый доступ они выложили в том числе и модели, которые были дообучены на нескольких задачах. Задача в модель передается через префикс (посмотрите на начало примеров выше). Эти модели есть на huggingface, давайте попробуем взять какую-то модель и попробовать сходу решить задачу саммаризации.

In [1]:
import pandas as pd
import numpy as np
import torch
import json

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
# MODEL_NAME = 't5-large'
MODEL_NAME = 't5-base'
# MODEL_NAME = 't5-small'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Возьмем какой-нибудь текст

In [5]:
task_prefix = "summarize: {}"

text = """
Badgers burrowing under rail tracks have halted trains in the northern and southern Netherlands, forcing lengthy cancellations on at least two lines.
All trains were halted Tuesday afternoon on a busy line between the southern cities of Den Bosch and Boxtel after the animals dug into a dike carrying rails. The national railway company said the line would be out of service for at least a week.
The digging means "the rails can subside and then the safety of train traffic can no longer be guaranteed," ProRail, the company that maintains the Dutch rail network said in a statement.
Earlier this month, badgers also burrowed under tracks near the northern village of Molkwerum in Friesland province, knocking a line out of service until next month while workers seek permission to shift the animals.
Badgers are protected animals in the Netherlands, so rail operators have to get permission to move them or disturb their habitat before repairs can begin.
"""



С моделями в huggingface удобнее всего работать через torch, но это не страшно, так как все основные вещи реализованы в transformers и они одинаковые для torch и tf. 

Попробуем сгенерировать саммари

In [6]:
inputs = tokenizer([task_prefix.format(text)], 
                    return_tensors="pt", padding=True)

output_sequences = model.generate(
    # this parameters are also important but you can read about them in the docs and just try changing them
    num_beams=5,
    max_length=100,
    no_repeat_ngram_size=3, 
#     repetition_penalty= 5.0,
#     length_penalty=0.01,
#     early_stopping=True,
#     do_sample=True, 
#     top_k=30, 
#     top_p=0.8, 
    early_stopping=True,
#     num_return_sequences=3,
    num_return_sequences= 1,
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,  # disable sampling to test if batching affects output
)


In [7]:
summaries = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [8]:
summaries

['all trains halted on a busy line between den Bosch and boxtel. badgers dug into a dike carrying rails. the national railway company says the line will be out of service for at least a week.']

Работает неплохо, но конечно для реального практическо применения нужно тюнить модель дополнительно

## FLAN

![](https://1.bp.blogspot.com/-_kPdaMrcRWI/YV2b-XFoRxI/AAAAAAAAIMw/KDjg0IfuoK8hjpSXNODoV46D8Rb5rK8hgCLcBGAsYHQ/w640-h178/image3.gif)

Второя статья - FLAN (тоже от Google Research, тоже огромная, Finetuned Language Models Are Zero-Shot Learners, https://arxiv.org/abs/2109.01652, середина 2021 года)

В этой статье уже заметен сдвиг в сторону общих моделей и уже сформировался подход к такому обучению через инструкции. Основная идея в статье - переделать различные NLP датасеты в большой датасет разнообразных инструкций (они сделали различные темплейты на правилах и прогнали их через размеченные датасеты) и обучить модель решать сразу всё. Инструкции при этом это не какие-то технические теги как в T5, а нормальные человеческие инструкции (буквально что-то вроде "Translate this text from English to Russian", "Write five topics that describe this text", "What is the sentiment of this text? Options: Negative, Positive, Neutral."). При таком подходе они заметили, что модель начинает обобщаться на инструкции, которых она никогда не видела - так как модель предобучена на большом количестве текстов, она уже хорошо понимает язык и экстраполирует инструкции из обучающей выборки, используя свое понимание языка). И чем больше таких инструкций, тем лучше получалось.

Они попробовали такой подход с разными моделями (T5, PALM) и везде получалось хорошо решать новые задачи.

FLAN варианты моделей также доступны на huggingface. Давайте попробуем с таким же текстом.

In [9]:
import pandas as pd
import numpy as np
import torch
import json

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [11]:
MODEL_NAME = 'google/flan-t5-small'

In [12]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Инструкции модели можно передавать в свободном формате, поэтому сделаем функцию, чтобы удобнее было пробовать разные инструкции.

In [13]:
def predict_for_instruction(instruction, text, model):
    

    inputs = tokenizer([instruction.format(text)], 
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=5,
        max_length=100,
        no_repeat_ngram_size=3, 
    #     repetition_penalty= 5.0,
    #     length_penalty=0.01,
    #     early_stopping=True,
    #     do_sample=True, 
    #     top_k=30, 
    #     top_p=0.8, 
        early_stopping=True,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
    return summaries[0]

In [14]:
text = """
Badgers burrowing under rail tracks have halted trains in the northern and southern Netherlands, forcing lengthy cancellations on at least two lines.
All trains were halted Tuesday afternoon on a busy line between the southern cities of Den Bosch and Boxtel after the animals dug into a dike carrying rails. The national railway company said the line would be out of service for at least a week.
The digging means "the rails can subside and then the safety of train traffic can no longer be guaranteed," ProRail, the company that maintains the Dutch rail network said in a statement.
Earlier this month, badgers also burrowed under tracks near the northern village of Molkwerum in Friesland province, knocking a line out of service until next month while workers seek permission to shift the animals.
Badgers are protected animals in the Netherlands, so rail operators have to get permission to move them or disturb their habitat before repairs can begin.
"""


In [15]:
instruction = "Give a summary of this text: {}"
predict_for_instruction(instruction, text, model)

'Badgers burrowing under rail tracks in the Netherlands have halted trains for at least a week.'

In [16]:
instruction = "Give a very short summary of this text: {}"
predict_for_instruction(instruction, text, model)

'Badgers burrowed under rail tracks in northern and southern Netherlands, forcing lengthy cancellations on at least two lines'

In [27]:
instruction = "Write a title for the following text:{}"
predict_for_instruction(instruction, text, model)

'Badgers halted in northern and southern Netherlands'

In [28]:
instruction = "Suggest keywords for this text. Text: {}"
predict_for_instruction(instruction, text, model)

'animal, stop, train'

## InstructGPT

FLAN модели работали хорошо, но все еще недостаточно. OpenAI довел их до состояния, когда их можно использовать на практике. Они публиковали несколько статей и описаний своих экспериментов:

https://openai.com/research/improving-language-model-behavior
https://openai.com/research/instruction-following
https://cdn.openai.com/papers/Training_language_models_to_follow_instructions_with_human_feedback.pdf
https://openai.com/research/learning-to-summarize-with-human-feedback

Они добавили еще одну важную часть - RLHF. Про нее мы попытаемся поговорить на следующем занятии. Пока сфокусируемся на инструкциях. Из описания OpenAI видно, что их подход очень похож на FLAN, но они машстабировали его и использовали для своих датасетов инструкции на основе реальных запросов к их API. И они продолжают это делать, исправляя все больше ошибок и нежелательных ответов. 
Также они сильно ускорились, когда добавили интерфейс (ChatGPT). Они даже говорили, что уже очень хорошая модель была доступна в их API около полугода и никто особо не обращал внимания на нее, хотя она уже работала как ChatGPT, но ей нужно было подавать правильный промпт. Когда они решили это через интерфейс (и промпт на бекенде), количество пользователей сильно увиличилось и к ним потекло очень много реальных запросов, на которых они быстро стали дообучаться.

Открытых моделей тут нет, поэтому перейдем к следующему шагу.

## Alpaca 

Разнообразние и естественность инструкция влияет на качество модели, но создавать такие датасеты сложно и дорого, а корпорации не делятся. Поэтому многие работы в обучении на инструкциях посвящены способам сгенерировать синтетические, но как можно более реалистичные датасеты. Значимая работа в этом направлении - Stanford Alpaca 
![](https://crfm.stanford.edu/static/img/posts/2023-03-13-alpaca/alpaca_main.jpg)

Код и датасет можно найти тут - https://github.com/tatsu-lab/stanford_alpaca
Дальше код взят из train.py и немного изменен

Авторы Альпаки дообучили модель LLaMA (7 миллиардов параметров) на датасете инструкций, который они сгенерировали с помощью OpenAI API и получилась модель, которая очень похожа по качеству на саму модель от OpenAI.   

LLaMa - это серия предобученных моделей от Meta. Они были опубликованы в 2023 году (LLaMA 1 в феврале, а LLaMA 2 в июле) и Meta утверждает, что по метрикам их меньшие модели сравнимы с GPT-3 (которая около 175 млрд параметров). LLaMA 2 долгое время держалась в топе открытых моделей, где конкуренцию ей составляют Mistral, DBRX, Grok-1.  


Незадолго до релиза LLaMA Meta сталкивалась с критикой за свою модель Galactica, которая была предобучена на научных статьях. Сначала они выложили её в открытый доступ, но быстро оказалась, что она может генерировать псевдонаучные и лженаучные тексты и Meta быстро закрыла доступ к этой модели. Поэтому модель LLaMA строго говоря не выложена в открытый доступ и имеет некомерческую лицензию. Чтобы скачать модель, нужно заполнять специальную форму и ждать пока ее одобрят. Но естественно люди, которые получили доступ к модели начали выкладывать ее в открытый доступ - например, кто-то делал ПР в либу Meta, в котором предлагается добавить в Readme.md ссылку на [торент](https://github.com/facebookresearch/llama/pull/73/commits/016a53608c5eae1021e171b9c4f06a9783fc14c0) 
LLaMA 2 также имеет некомерческую лицензию и требует заявки на лиценцию, однако одобрение лицензии занимает небольшое время, поэтому модель уже воспринимается как полностью открытая.

Датасет инструкций Alpaca сгенерировали на основе статьи - https://arxiv.org/abs/2212.10560 И как они говорят у них ушло около 500$ на все, что в тысячи раз дешевле того, что, предполагается, потратил сам OpenAI на свои модели. Но OpenAI запрещают использовать свои модели в таких целях и поэтому итоговую модель Alpaca они пока не выкладывают.

Но они выложили в открытый доступ датасет и можно самому попробовать дообучить какую-то открытую предобученную модель. Коммерческая применимость такой модели, однако, все еще под вопросом. 

Скачаем датасет

In [2]:
!wget https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json

--2023-03-21 21:03:20--  https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22773992 (22M) [text/plain]
Saving to: ‘alpaca_data.json’

alpaca_data.json    100%[===================>]  21.72M  38.9MB/s    in 0.6s    

2023-03-21 21:03:21 (38.9 MB/s) - ‘alpaca_data.json’ saved [22773992/22773992]



In [1]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

# import utils

Посмотрим на датасет.

In [2]:
data_alpaca = json.load(open('alpaca_data.json'))

In [3]:
data_alpaca[:3]

[{'instruction': 'Give three tips for staying healthy.',
  'input': '',
  'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'},
 {'instruction': 'What are the three primary colors?',
  'input': '',
  'output': 'The three primary colors are red, blue, and yellow.'},
 {'instruction': 'Describe the structure of an atom.',
  'input': '',
  'output': 'An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.'}]

В нем каждый пример это инструкция, опциональный контекст и ответ.
Для модели эти примеры еще оборачиваются в специальный промпт, который говорит модели, что она должна следовать инструкциям.

In [4]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}


Давайте попробуем дообучить модель от facebook - opt (она открытыя и устроена как LLama и GPT - это декодер онли модель)

Далее код взят из гитхаба Alpaca и он на торче, но если поизучать его, то будет видно, что тут происходят те же манипуляции, что мы делали раньше (превращение токенов в индексы и паддинг/урезание последовательностей)

In [5]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [6]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

Далее это оборачивается к классы, которые предобрабатывают данные к формату huggingface.

In [7]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = json.load(open(data_path))

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

Загружаем модель

In [8]:
# model_name = 'facebook/opt-350m'
model_name = "facebook/opt-125m"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=512,
        cache_dir="huggingface_cache",
    )

In [9]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

Токенизируем данные

In [11]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path="alpaca_data.json")
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)


Задаем параметры обуечения

In [12]:
train_args = transformers.TrainingArguments(learning_rate=1e-5, 
                 num_train_epochs=1,
                 per_device_train_batch_size=2,
                 gradient_accumulation_steps=1,
                 evaluation_strategy='no',
                 weight_decay=0.,
                 warmup_ratio=0.03,
                 lr_scheduler_type="cosine",
                 save_strategy='no',
                 logging_steps=1000,
                 output_dir="opt125_instruct_ft")

И обучаем

In [13]:
trainer = Trainer(model=model, 
                 tokenizer=tokenizer, 
                 args=train_args,
                 train_dataset=train_dataset, 
                 eval_dataset=None, 
                 data_collator=data_collator)

In [14]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,2.332500
2000,2.230100
3000,2.162800
4000,2.120400
5000,2.141000
6000,2.136100
7000,2.130600
8000,2.094200
9000,2.068500
10000,2.066400


TrainOutput(global_step=26001, training_loss=2.066676487524158, metrics={'train_runtime': 2371.6488, 'train_samples_per_second': 21.927, 'train_steps_per_second': 10.963, 'total_flos': 3745819289088000.0, 'train_loss': 2.066676487524158, 'epoch': 1.0})

Сохраним модель

In [16]:
trainer.save_model('opt125_ft_02')

И давайте попробуем ее на том же тексте

In [91]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [92]:
MODEL_NAME = 'opt125_ft_02'

In [93]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512, max_length=512)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, max_length=512)

In [94]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Below is an instruction that describes a task, paired with an input that provides further context. "
              "Write a response that appropriately completes the request.\n\n"
              f"### Instruction:\n{instruction}\n\n### Input:\n{text}\n\n### Response:")

    inputs = tokenizer([prompt], 
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=1,
#         temperature=0.4,
#         max_length=100,
        max_new_tokens=20,
#         no_repeat_ngram_size=3,
    #     repetition_penalty= 5.0,
    #     length_penalty=0.01,
    #     early_stopping=True,
    #     do_sample=True, 
    #     top_k=30, 
    #     top_p=0.8, 
        early_stopping=True,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

In [95]:
text = """
Badgers burrowing under rail tracks have halted trains in the northern and southern Netherlands, forcing lengthy cancellations on at least two lines.
All trains were halted Tuesday afternoon on a busy line between the southern cities of Den Bosch and Boxtel after the animals dug into a dike carrying rails. The national railway company said the line would be out of service for at least a week.
The digging means "the rails can subside and then the safety of train traffic can no longer be guaranteed," ProRail, the company that maintains the Dutch rail network said in a statement.
Earlier this month, badgers also burrowed under tracks near the northern village of Molkwerum in Friesland province, knocking a line out of service until next month while workers seek permission to shift the animals.
Badgers are protected animals in the Netherlands, so rail operators have to get permission to move them or disturb their habitat before repairs can begin.
"""

In [96]:
instruction = "Give a summary of this text."
predict_for_instruction(instruction, text, model)

'Badgers burrowed under the tracks of the northern and southern Netherlands, causing delays on the two'

In [97]:
instruction = "Give a very short summary of this text."
predict_for_instruction(instruction, text, model)

'Badgers burrowed under the tracks of the northern and southern Netherlands, causing delays on the two'

In [98]:
instruction = "Write a headline for the following text."
predict_for_instruction(instruction, text, model)

'Badgers burrow under rail tracks in northern Netherlands, forcing lengthy cancellations on at least two lines'

In [100]:
instruction = "Suggest a headline for this text."
predict_for_instruction(instruction, text, model)

'Badgers burrow under rail tracks: halt trains in northern and southern Netherlands, forcing lengthy cancellations'

## Dolly

Единственный действительно открытый и от руки написанный датасет инструкций - это databricks/databricks-dolly-15k. Этот датасет вручную написали сотрудники компании Databricks. Он выложен в открытый доступ и его разрешено использовать в коммерческих целях. 
В тот момент казалось, что таких датасетов станет гораздо больше. На основе долли/альпаки быстро файюнтинились открытые предобученные модели и они даже неплохо работали, но почему-то хороших вручную написанных датасетов больше никто не опубликовал. Возможно это просто очень дорого, или же компании поняли, что на этом можно заработать и лучше не делится с конкурентами. Новые модели часто выходят уже с instruct fine-tuned версиями, но данные никто не раскрывает (например, в релизе `databricks/dbrx-instruct` вообще нет даже описания использованного датасета!)

In [29]:
from datasets import load_dataset

In [30]:
data_dolly = load_dataset("databricks/databricks-dolly-15k")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [31]:
data_dolly

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

In [34]:
data_dolly['train'][12]

{'instruction': "From the passage list down the areas for which Dar es Salaam is Tanzania's most prominent city. List the results in comma separated format.",
 'context': "Dar es Salaam (/ˌdɑːr ɛs səˈlɑːm/; from Arabic: دَار السَّلَام, romanized: Dâr es-Selâm, lit.\u2009'Abode of Peace') or commonly known as Dar, is the largest city and financial hub of Tanzania. It is also the capital of Dar es Salaam Region. With a population of over six million people, Dar is the largest city in East Africa and the seventh-largest in Africa. Located on the Swahili coast, Dar es Salaam is an important economic centre and is one of the fastest-growing cities in the world.\n\nThe town was founded by Majid bin Said, the first Sultan of Zanzibar, in 1865 or 1866. It was the main administrative and commercial center of German East Africa, Tanganyika, and Tanzania. The decision was made in 1974 to move the capital to Dodoma and was officially completed in 1996.\n\nDar es Salaam is Tanzania's most prominent

## Madlad-400

Подход предложенный в T5 не обязательно переносить на общие инструкции. В конце концов нам нужны не только чат-боты общего назначения, но и более специализированные модели. В статье 2023 года [MADLAD-400: A Multilingual And Document-Level
Large Audited Dataset](https://arxiv.org/pdf/2309.04662.pdf) предлагается способ обучить многоязычную переводную модель на основе T5 архитектуры. Только вместо ключевых тегов для инструкции здесь используются теги для целевых языков - `<2ru>, <2en>, <2de>...`. Всего поддерживается около 400 языков и очень большое количество языковых пар. За счет обучения на всех данных сразу модель может обобщаться на языковые пары, которые в обучающей выборке могли не встречаться. Стандартный подход для моделей машинного обучения - это обучать по две модели на языковую пару (en-ru и ru-en например). При таком подходе для многих пар не получится обучить модель. Это можно решать через перевод на промежуточный язык (обычно английский) и последующий перевод на целевой язык доступной моделью, но это вносит дополнительный ошибки на счет двойного перевода. В Madlad эта логика реализована как бы внутри модели. У нас может не быть переводов с башкирского на испанский, но модель видела все доступные примеры с этими языками в других парах и этого может хватить для генерации zero-shot перевода. Этому также помогает то, что модель учится на всех языках сразу и возможно выучивает общеязыковые паттерны. 

Еще такую модель удобно применять, когда в изначальном тексте намешаны слова из different языков.

In [4]:
#pip install accelerate

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = 'jbochi/madlad400-3b-mt'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/11.8G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/830 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.43M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.6M [00:00<?, ?B/s]

In [3]:
text = "<2en> Еще такую модель удобно применять, когда в изначальном тексте намешаны слова из different языков."
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
outputs = model.generate(input_ids=input_ids)

tokenizer.decode(outputs[0], skip_special_tokens=True)

'This model is also useful when the original text contains words from different languages.'